In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, Markdown, SVG, HTML
import pandas as pd
import altair as alt
import re
import pickle
from utils import seconds_to_ms, ms_to_seconds
from benchmark_result import get_benchmark_results, BenchmarkType, SchedulerType, get_broadcast_source, get_async_source
from benchmarks.utils import echo
from benchmarks.throughput import make_benchmark, make_multiple_message_benchmark

In [2]:
#benchmark_results = get_benchmark_results()
from benchmark_result import BenchmarkResult, Result 
with open('saved_results.pkl', 'rb') as saved_results:
    benchmark_results = pickle.load(saved_results)

# time_async_messages

This benchmark comes from benchmarking the runtime of sending a large amount of messages asynchronously and after sending them all, waiting for the reply

In [3]:
??make_multiple_message_benchmark

Signature: make_multiple_message_benchmark(get_view)
Docstring: <no docstring>
Source:   
def make_multiple_message_benchmark(get_view):
    class AsyncMessagesSuite:
        param_names = ['Number of engines', 'number_of_messages']
        timer = timeit.default_timer
        timeout = 180
        params = [engines, number_of_messages]

        view = None
        client = None
        reply = None

        def setup(self, number_of_engines, number_of_messages):
            self.client = ipp.Client(profile='asv')
            self.view = get_view(self)
            self.view.targets = list(range(number_of_engines))

            wait_for(lambda: len(self.client) >= number_of_engines)

        def time_async_messages(self):
            replies = []
            for i in range(20):
                reply = self.view.apply_async(
                    echo, np.array([0] * 1000, dtype=np.int8)
                )
                replies.append(reply)
            for reply in replies:
             

## Results from benchmarking on 64 cores


In [4]:
source = get_async_source(benchmark_results)
dview = pd.DataFrame(source['DirectView']) 
dview['Scheduler name'] = 'DirectView'
dview['Speedup'] = 1


In [5]:
datas = []
for scheduler_name, scheduler_results in source.items():
    data = pd.DataFrame(scheduler_results) 
    data['Scheduler name'] = scheduler_name
    data['Speedup'] = 1 if scheduler_name == 'DirectView' else dview['Duration in ms'] / data['Duration in ms']
    datas.append(data)
data = pd.concat(datas)
data

,Duration in ms,Number of messages,Number of engines,Scheduler name,Speedup
0,4.75,1,1,DirectView,1.000000
1,17.70,5,1,DirectView,1.000000
2,29.59,10,1,DirectView,1.000000
3,58.47,20,1,DirectView,1.000000
4,133.31,50,1,DirectView,1.000000
...,...,...,...,...,...
49,593.23,1,1024,NonCoalescing,5.581141
50,2398.68,5,1024,NonCoalescing,2.804805
51,4529.79,10,1024,NonCoalescing,2.870884
52,8927.69,20,1024,NonCoalescing,NaN


In [6]:
filtered = data[(data['Number of messages'] == 20) & (data['Number of engines'] == 1)]
filtered

,Duration in ms,Number of messages,Number of engines,Scheduler name,Speedup
3,58.47,20,1,DirectView,1.000000
3,59.19,20,1,Coalescing,0.987836
3,60.54,20,1,NonCoalescing,0.965808


In [7]:
data['Messages per engine per second'] = round(data['Number of messages'] / data['Duration in ms']  * 1000, 2)
data['Total messages per second'] = round(((data['Number of messages'] * data['Number of engines']) / data['Duration in ms']) * 1000, 2)

In [8]:
data

,Duration in ms,Number of messages,Number of engines,Scheduler name,Speedup,Messages per engine per second,Total messages per second
0,4.75,1,1,DirectView,1.000000,210.53,210.53
1,17.70,5,1,DirectView,1.000000,282.49,282.49
2,29.59,10,1,DirectView,1.000000,337.95,337.95
3,58.47,20,1,DirectView,1.000000,342.06,342.06
4,133.31,50,1,DirectView,1.000000,375.07,375.07
...,...,...,...,...,...,...,...
49,593.23,1,1024,NonCoalescing,5.581141,1.69,1726.14
50,2398.68,5,1024,NonCoalescing,2.804805,2.08,2134.51
51,4529.79,10,1024,NonCoalescing,2.870884,2.21,2260.59
52,8927.69,20,1024,NonCoalescing,NaN,2.24,2293.99


In [16]:
dview = data[data['Number of messages'] == 100]
dview['Number of engines'].unique()


array([   1,    2,   16,   64,  128,  256,  512, 1024])

In [17]:
alt.Chart(dview).mark_line(point=True).encode(
    alt.X(
        'Number of engines',
        scale=alt.Scale(type='log', base=2)
    ),
    alt.Y(
        'Messages per engine per second',
        scale=alt.Scale(type='log')
    ),
    color='Scheduler name:N',
    tooltip='Messages per engine per second',
).configure_axis(labelFontSize=20, titleFontSize=20).properties(title='Runtime of apply using DirectView', width=1080).interactive().display(renderer='svg')

alt.Chart(...)

In [18]:
alt.Chart(dview).mark_line(point=True).encode(
    alt.X(
        'Number of engines',
        scale=alt.Scale(type='log', base=2)
    ),
    alt.Y(
        'Total messages per second',
        scale=alt.Scale(type='log')
    ),
    color='Scheduler name:N',
    tooltip='Total messages per second',
).configure_axis(labelFontSize=20, titleFontSize=20).properties(title='Runtime of apply using DirectView', width=1080).interactive().display(renderer='svg')

alt.Chart(...)

In [25]:
for scheduler_name in data['Scheduler name'].unique():
    scheduler_data = data[data['Scheduler name'] == scheduler_name]
    alt.Chart(scheduler_data).mark_line(point=True).encode(
        alt.X(
            'Number of messages',
            scale=alt.Scale(type='log')
        ),
        alt.Y(
            'Messages per engine per second',
        ),
        color='Number of engines:N',
        tooltip='Duration in ms',    
    ).configure_axis(labelFontSize=20, titleFontSize=20).properties(title=scheduler_name, width=800).interactive().display(renderer='svg')

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [17]:
data['combined']=  data['Scheduler name'] + ' ' + data['Number of engines'].astype(str)
alt.Chart(data[data['Scheduler name'] != 'DirectView']).mark_line(point=True).encode(
    alt.X(
        'Number of messages',
            scale=alt.Scale(type='log')
    ),
    alt.Y(
        'Speedup',
    ),
    color='Number of engines:N',
    strokeDash=alt.StrokeDash(shorthand='Scheduler name', legend=None),
    tooltip='combined',

).properties(title='schedulers vs directView scaling engines', width=800).interactive().display(renderer='svg')


alt.Chart(...)

In [10]:
for engine in data['Number of engines'].unique():    
    alt.Chart(data[data['Number of engines'] == engine]).mark_bar().encode(
        x='Scheduler name',
        y='Duration in ms',
        color='Scheduler name:N',
        column='Number of messages:N',    
        tooltip='Duration in ms'
    ).properties(title=f'Runtime on {engine} engines:').interactive().display(renderer='svg')

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)